## Before training

This program saves the last 3 generations of models to Google Drive. Since 1 generation of models is >1GB, you should have at least 3GB of free space in Google Drive. If you do not have such free space, it is recommended to create another Google Account.

Training requires >10GB VRAM. (T4 should be enough) Inference does not require such a lot of VRAM.

## Installation

In [ ]:
#@title Check GPU
!nvidia-smi

In [ ]:
#@title Install dependencies
!python -m pip install -U pip wheel
!pip install -U ipython so-vits-svc-fork

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

## Training

In [ ]:
#@title Make dataset directory
!cd ./content

In [ ]:
!mkdir -p "dataset_raw/44k"

In [ ]:
#!rm -r "dataset_raw/44k"
#!rm -r "dataset/44k"

In [ ]:
#@title Copy your dataset
DATASET_NAME = "kiritan" #@param {type: "string"}
!cp -R /content/drive/MyDrive/so-vits-svc-fork/dataset/{DATASET_NAME}/ -t "dataset_raw/44k/"

In [ ]:
#@title Download dataset (Tsukuyomi-chan JVS)
#@markdown Make sure you agree to the license when using this dataset.
# !wget https://tyc.rei-yumesaki.net/files/sozai-tyc-corpus1.zip
# !unzip sozai-tyc-corpus1.zip
# !mv "/content/つくよみちゃんコーパス Vol.1 声優統計コーパス（JVSコーパス準拠）/おまけ：WAV（+12dB増幅＆高音域削減）/WAV（+12dB増幅＆高音域削減）" "dataset_raw/44k/tsukuyomi"

In [ ]:
#@title Automatic preprocessing
!svc pre-resample

In [ ]:
!svc pre-config

In [ ]:
#@title Copy configs file
!cp configs/44k/config.json drive/MyDrive/so-vits-svc-fork

In [ ]:
!svc pre-hubert

In [ ]:
#@title Train
#!svc train
!svc train --model-path drive/MyDrive/so-vits-svc-fork/logs/44k

## Inference

In [ ]:
#@title Get the author's voice as a source
!wget "https://github.com/34j/34j/raw/main/jvs-parallel100/99.wav"

In [ ]:
#@title Use trained model
#!svc infer 100.wav --speaker kiritan -m logs/44k/G_1600.pth
!svc infer 99.wav -t 16 --speaker kiritan -m drive/MyDrive/so-vits-svc-fork/logs/44k/G_4000.pth -c drive/MyDrive/so-vits-svc-fork/logs/44k/config.json

In [ ]:
#@title Use pretrained model
!wget -O "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Pinkie%20(speaking%20sep)/G_166400.pth"
!wget -O "https://huggingface.co/therealvul/so-vits-svc-4.0/resolve/main/Pinkie%20(speaking%20sep)/config.json"

In [ ]:

!svc infer 100.wav --speaker "Pinkie {neutral}" -m G_166400.pth -c config.json